In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [5]:
both_gender_1960 = pd.read_csv('data/both_gender/1960.csv')
both_gender_1960

,Origin_name,Origin_code,Gender_Code,Destination_name,Destination_code,1960
0,Afghanistan,AFG,TOT,Afghanistan,AFG,0
1,Afghanistan,AFG,TOT,Albania,ALB,0
2,Afghanistan,AFG,TOT,Algeria,DZA,41
3,Afghanistan,AFG,TOT,American Samoa,ASM,0
4,Afghanistan,AFG,TOT,Andorra,AND,0
...,...,...,...,...,...,...
53587,Zimbabwe,ZWE,TOT,Wallis and Futuna,WLF,0
53588,Zimbabwe,ZWE,TOT,West Bank and Gaza,PSE,0
53589,Zimbabwe,ZWE,TOT,"Yemen, Rep.",YEM,0
53590,Zimbabwe,ZWE,TOT,Zambia,ZMB,90416
